# Debugging 1D solver failures

This notebook shows an example where the 1D solver fails, and demonstrates the use of callbacks to diagnose the failure modes.
This example is adapted from the bug report at https://github.com/Cantera/cantera/issues/384.

In [ ]:
import cantera as ct
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.constrained_layout.use'] = True

print(f"Running Cantera version {ct.__version__}")

### parameter values

In [ ]:
p = 50 * ct.one_atm  # pressure
tburner = 300.0  # burner temperature
mdot = 0.22  # kg/m^2/s
width = 0.05  # m

### Set up the problem

In [ ]:
gas = ct.Solution('h2o2.cti')

# set state to that of the unburned gas at the burner
gas.set_equivalence_ratio(0.4, 'H2', 'O2:1.0, AR:4')
gas.TP = tburner, p

sim = ct.BurnerFlame(gas=gas, width=width)
sim.burner.mdot = mdot
sim.set_refine_criteria(ratio=3, slope=0.1, curve=0.2, prune=0)

### Try to solve it...

In [ ]:
sim.solve(loglevel=1, auto=False)

### That didn't work...what went wrong?
* We can see that the solver failed because it was taking too many timesteps without those timesteps increasing in size, as would normally be expected.
* Add a callback function to collect data after each timestep
* We are only interested in the timesteps leading up to the final, failed solution attempt, but we don't know when that will occur.
* Use the `time_step` callback to collect data after each timestep, no matter what.
* Use the `steady` callback to clear data after a successful solve 

In [ ]:
# variables used to capture data within the callback functions
grid = []
TT = []
YY = []
UU = []

# Callback function must take one argument, even if we don't use it
def time_step_callback(_):
    grid[:] = sim.grid
    TT.append(sim.T)
    YY.append(sim.Y[gas.species_index('O2')])
    UU.append(sim.u)
    return 0  # Callback function must return a number

def steady_callback(_):
    TT.clear()
    YY.clear()
    UU.clear()
    return 0

# Recreate the flame object and attach the callback functions
gas.set_equivalence_ratio(0.4, 'H2', 'O2:1.0, AR:4')
gas.TP = tburner, p
sim = ct.BurnerFlame(gas=gas, width=width)
sim.burner.mdot = mdot
sim.set_refine_criteria(ratio=3, slope=0.1, curve=0.2, prune=0)

sim.set_time_step_callback(time_step_callback)
sim.set_steady_callback(steady_callback)

### Attempt to solve the flame again
Wrap the call to `solve` in a `try`/`except` block, since we are expecting it to fail

In [ ]:
try:
    sim.solve(loglevel=0, auto=False)
except ct.CanteraError as e:
    print(e)
    sim.show_stats()

## Convert saved data to 2D arrays

In [ ]:
T = np.array(TT)
Y = np.array(YY)
U = np.array(UU)

## Plot profiles


In [ ]:
f,ax = plt.subplots(1, 1, figsize=(5,4))
# Plot the last N profiles
N = 50  
# Use a sequential color map so we can tell which lines belong to which timestep
C = plt.cm.viridis(np.linspace(0, 1, N))
for i in range(N):
    ax.plot(grid, T[len(T)-N+i], '.-', color=C[i])
ax.set_xlim(0, 0.002);

### Let's look at what's happening as a function of time

In [ ]:
f,ax = plt.subplots(1, 3, figsize=(9, 3.5))
ax[0].plot(T[:,1])
ax[1].plot(Y[:,1])
ax[2].plot(U[:,1])
ax[0].set_title('Temperature at $j=1$')
ax[1].set_title('$Y_{O_2}$ at $j=1$')
ax[2].set_title('velocity at $j=1$');